In [1]:
import pandas as pd
import numpy as np

In [2]:
support_data = pd.read_csv('supporting_master_data.csv')
support_data.head()

,SK_ID_CURR,BUREAU_<LAMBDA>,BUREAU_SUM,BUREAU_MEAN,BUREAU_SUM.1,BUREAU_MEAN.1,BUREAU_MAX,BUREAU_MEAN.2,POS_MIN,POS_MAX,...,PREV_MEAN.2,PREV_MEAN.3,PREV_MAX.2,PREV_MIN,INST_SUM,INST_MEAN,INST_SUM.1,INST_MEAN.1,INST_MAX,INST_MIN
0,100001,Closed,1453365.000,207623.571429,596686.5,85240.928571,0,0.0,0.0,4.0,...,2520.0,0.104326,-1740.0,-1740.0,41195.925,5885.132143,41195.925,5885.132143,-1628.0,-2916.0
1,100002,Closed,865055.565,108131.945625,245781.0,49156.200000,0,0.0,6.0,24.0,...,0.0,0.000000,-606.0,-606.0,219625.695,11559.247105,219625.695,11559.247105,-49.0,-587.0
2,100003,Closed,1017400.500,254350.125000,0.0,0.000000,0,0.0,0.0,12.0,...,3442.5,0.050030,-746.0,-2341.0,1618864.650,64754.586000,1618864.650,64754.586000,-544.0,-2324.0
3,100004,Closed,189037.800,94518.900000,0.0,0.000000,0,0.0,0.0,4.0,...,4860.0,0.212008,-815.0,-815.0,21288.465,7096.155000,21288.465,7096.155000,-727.0,-795.0
4,100005,Active,657126.000,219042.000000,568408.5,189469.500000,0,0.0,0.0,12.0,...,4464.0,0.108964,-315.0,-757.0,56161.845,6240.205000,56161.845,6240.205000,-470.0,-736.0


In [3]:
# Outlier detection using 1.5 * IQR rule
outliers = []

# Loop over numeric features
for column in support_data.select_dtypes(include=[np.number]).columns:
    col_data = support_data[column].dropna()  # Drop missing values for IQR calculation
    
    if col_data.empty:
        continue  # Skip columns with no data
    
    # Calculate IQR
    q1 = col_data.quantile(0.25)
    q3 = col_data.quantile(0.75)
    iqr = q3 - q1  # Interquartile range
    
    # Define bounds
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Identify outliers
    outlier_mask = (col_data < lower_bound) | (col_data > upper_bound)
    
    # Explicitly calculate scalar outlier count
    outlier_count = int(outlier_mask.astype(int).sum())  # Convert mask to int and sum
    
    # Append outlier statistics
    outliers.append({
        'Feature': column,
        'Outlier Count': outlier_count,
        'Outlier Percentage': (outlier_count / len(col_data)) * 100  # Use non-NaN count
    })

# Convert to DataFrame
outliers_df = pd.DataFrame(outliers)

# Sort by 'Outlier Percentage'
outliers_df = outliers_df.sort_values(by='Outlier Percentage', ascending=False)

# Display the summary
print("\nOutliers Summary:")
print(outliers_df)


Outliers Summary:
          Feature  Outlier Count  Outlier Percentage
16       CC_MAX.1          18218           20.257978
17      CC_MEAN.2          18218           20.257978
11     POS_MEAN.1          54742           18.887754
10      POS_MAX.1          54742           18.887754
31       INST_MAX          30937           10.605868
8         POS_MAX          28353            9.783441
4   BUREAU_MEAN.1          28995            9.748217
3    BUREAU_SUM.1          29405            9.615416
2     BUREAU_MEAN          28035            9.167487
27       INST_SUM          26647            9.134945
7         POS_MIN          25177            8.687536
29     INST_SUM.1          25037            8.583016
1      BUREAU_SUM          25564            8.359412
28      INST_MEAN          24022            8.235258
25     PREV_MAX.2          23652            8.125294
22       PREV_SUM          22375            7.686600
30    INST_MEAN.1          22164            7.598113
15      CC_MEAN.1          

In [4]:
# Check for duplicates
num_duplicates = support_data.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Check for missing values (exact count and percentage)
missing_values = support_data.isnull().sum()
missing_percent = (missing_values / len(support_data)) * 100

missing_df = pd.DataFrame({
    'Feature': support_data.columns,
    'Missing Count': missing_values,
    'Missing Percentage': missing_percent
}).reset_index(drop=True)

print("\nMissing Values Summary:")
print(missing_df.sort_values(by='Missing Percentage', ascending=False))

Number of duplicate rows: 0

Missing Values Summary:
            Feature  Missing Count  Missing Percentage
16        CC_MEAN.1         242922           79.435338
17         CC_MAX.1         215881           70.592948
18        CC_MEAN.2         215881           70.592948
15           CC_SUM         215881           70.592948
14           CC_MAX         215881           70.592948
13          CC_MEAN         215881           70.592948
25      PREV_MEAN.3          31577           10.325659
24      PREV_MEAN.2          31577           10.325659
8           POS_MIN          16005            5.233625
9           POS_MAX          16005            5.233625
10         POS_MEAN          16005            5.233625
12       POS_MEAN.1          15983            5.226431
11        POS_MAX.1          15983            5.226431
19        PREV_MEAN          15160            4.957310
20         PREV_MAX          15160            4.957310
22       PREV_MAX.1          14720            4.813431
26       PRE

In [5]:
# Define the threshold for missing percentage
threshold = 70.0

# Identify columns to drop
columns_to_drop = support_data.loc[:, (support_data.isnull().mean() * 100) > threshold].columns

# Drop the columns from the DataFrame
support_data_cleaned = support_data.drop(columns=columns_to_drop)

# Display summary of dropped columns
print(f"Dropped Columns (More than {threshold}% Missing Values):")
print(columns_to_drop)

# Check remaining columns and their missing value percentages
remaining_missing_summary = support_data_cleaned.isnull().sum() / len(support_data_cleaned) * 100
print("\nRemaining Missing Values Summary:")
print(remaining_missing_summary[remaining_missing_summary > 0])

Dropped Columns (More than 70.0% Missing Values):
Index(['CC_MEAN', 'CC_MAX', 'CC_SUM', 'CC_MEAN.1', 'CC_MAX.1', 'CC_MEAN.2'], dtype='object')

Remaining Missing Values Summary:
BUREAU_MEAN       0.000654
BUREAU_MEAN.1     2.737639
POS_MIN           5.233625
POS_MAX           5.233625
POS_MEAN          5.233625
POS_MAX.1         5.226431
POS_MEAN.1        5.226431
PREV_MEAN         4.957310
PREV_MAX          4.957310
PREV_MEAN.1       4.813431
PREV_MAX.1        4.813431
PREV_SUM          4.813431
PREV_MEAN.2      10.325659
PREV_MEAN.3      10.325659
PREV_MAX.2        4.813431
PREV_MIN          4.813431
INST_SUM          4.612980
INST_MEAN         4.615269
INST_SUM.1        4.612980
INST_MEAN.1       4.612980
INST_MAX          4.615269
INST_MIN          4.615269
dtype: float64


In [6]:
# Identify numerical and categorical columns
numerical_columns = support_data_cleaned.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = support_data_cleaned.select_dtypes(include=['object']).columns

# Median imputation for numerical columns with missing values
for col in numerical_columns:
    if support_data_cleaned[col].isnull().sum() > 0:
        support_data_cleaned[col].fillna(support_data_cleaned[col].median(), inplace=True)

# Mode imputation for categorical columns with missing values (if any)
for col in categorical_columns:
    if support_data_cleaned[col].isnull().sum() > 0:
        support_data_cleaned[col].fillna(support_data_cleaned[col].mode()[0], inplace=True)

# Verify that there are no more missing values
print("\nRemaining Missing Values After Imputation:")
print(support_data_cleaned.isnull().sum().sum())


Remaining Missing Values After Imputation:
0


In [7]:
import numpy as np

# Initialize a dictionary to store outlier statistics
outliers = []

# Iterate over each numerical column to identify outliers using IQR
for column in support_data_cleaned.select_dtypes(include=['float64', 'int64']).columns:
    col_data = support_data_cleaned[column].dropna()  # Drop NaNs to avoid issues
    
    # Calculate Q1, Q3, and IQR
    Q1 = np.percentile(col_data, 25)
    Q3 = np.percentile(col_data, 75)
    IQR = Q3 - Q1
    
    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outlier_mask = (col_data < lower_bound) | (col_data > upper_bound)
    outlier_count = outlier_mask.sum()  # Count of outliers
    outlier_percentage = (outlier_count / len(col_data)) * 100  # Outlier percentage
    
    # Append the statistics to the list
    outliers.append({
        'Feature': column,
        'Outlier Count': outlier_count,
        'Outlier Percentage': outlier_percentage
    })

# Convert the outliers list into a DataFrame
outliers_df = pd.DataFrame(outliers)

# Display the sorted results by outlier percentage
print("\nOutliers Summary After Missing Value Handling:")
print(outliers_df.sort_values(by='Outlier Percentage', ascending=False))


Outliers Summary After Missing Value Handling:
          Feature  Outlier Count  Outlier Percentage
11     POS_MEAN.1          54742           17.900599
10      POS_MAX.1          54742           17.900599
25       INST_MAX          32972           10.781823
4   BUREAU_MEAN.1          30189            9.871784
21       INST_SUM          29557            9.665120
3    BUREAU_SUM.1          29405            9.615416
8         POS_MAX          28353            9.271413
2     BUREAU_MEAN          28035            9.167427
23     INST_SUM.1          27874            9.114780
22      INST_MEAN          26357            8.618722
15     PREV_MAX.1          26125            8.542858
1      BUREAU_SUM          25564            8.359412
7         POS_MIN          25177            8.232863
19     PREV_MAX.2          25172            8.231228
16       PREV_SUM          24907            8.144573
24    INST_MEAN.1          24523            8.019005
17    PREV_MEAN.2          22737            7.43498

In [9]:
# Save the dataset without handling outliers
support_data_cleaned.to_csv("supporting_master_data_raw.csv", index=False)

print("Not handled outliers version saved as 'supporting_master_data_raw.csv'.")

Not handled outliers version saved as 'supporting_master_data_raw.csv'.
